In [1]:
import numpy as np
import itertools

N = 10 ## Cambiar lectura de matriz
k = N-1
covmat = np.random.rand(N,N) #Aca corresponde ingresar la matriz de covarianzas ya calculada
covmat[[range(0,N)],[range(0,N)]] = 1 #Esta linea esta demas, eliminar cuando se vea como recibir la matriz desde matlab
arr = list(range(0,N)) #Arreglo para generar las mascaras de las submatrices cuadradas

In [2]:
arr

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [3]:
from ipyparallel import Client
rc = Client(profile='brincolab-cluster')
rc.purge_everything()  #Limpiar la sesion

/opt/miniconda/envs/jupyterhub/lib/python3.6/site-packages/ipyparallel/client/client.py:459: RuntimeWarning: 
            Controller appears to be listening on localhost, but not on this machine.
            If this is true, you should specify Client(...,sshserver='you@brincolab1')
            or instruct your controller to listen on an external IP.
  RuntimeWarning)


In [4]:
dview = rc[:] # use all engines

In [5]:
dview['covmat'] = covmat #Distribuimos la matriz de covarianza

In [6]:
dview.block = True #Modo block

In [7]:
import scipy.io as sio

In [10]:
indexes = sio.loadmat('int_list.mat')

In [11]:
indexes

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Thu Aug 29 20:41:57 2019',
 '__version__': '1.0',
 '__globals__': [],
 'int_list': array([[array([[1, 2]], dtype=uint8)],
        [array([[1, 3]], dtype=uint8)],
        [array([[1, 4]], dtype=uint8)],
        ...,
        [array([[ 1,  2,  4,  5,  6,  7,  8,  9, 10]], dtype=uint8)],
        [array([[ 1,  3,  4,  5,  6,  7,  8,  9, 10]], dtype=uint8)],
        [array([[ 2,  3,  4,  5,  6,  7,  8,  9, 10]], dtype=uint8)]],
       dtype=object)}

In [12]:
indexes_list = indexes['int_list']

In [63]:
indexes_list

array([[array([[1, 2]], dtype=uint8)],
       [array([[1, 3]], dtype=uint8)],
       [array([[1, 4]], dtype=uint8)],
       ...,
       [array([[ 1,  2,  4,  5,  6,  7,  8,  9, 10]], dtype=uint8)],
       [array([[ 1,  3,  4,  5,  6,  7,  8,  9, 10]], dtype=uint8)],
       [array([[ 2,  3,  4,  5,  6,  7,  8,  9, 10]], dtype=uint8)]],
      dtype=object)

In [21]:
with dview.sync_imports():
    import numpy

importing numpy on engine(s)


In [22]:
dview.scatter('indexes_list',indexes_list)

1008

1012

In [59]:
def det():
    N = len(indexes_list)
    dets = numpy.zeros(N,dtype=numpy.float32)
    if len(indexes_list) > 0:
        idx_mat = indexes_list
    else:    
        return {'dets':[],'matrices':[],'idx':[]}
    matrices = []
    idxs = []
    i=0
    for ind in idx_mat:
        idx = ind[0][0] - 1
        #idxs.append(idx)
        matrix = covmat[idx][:,idx]
        dets[i] = numpy.linalg.det(matrix)
        matrices.append(matrix)
        idxs.append(idx)
        i += 1
    return {'dets':dets,'idx':idxs}

In [60]:
ar = dview.apply_async(det)
ar.wait()
responses = ar.get()

In [62]:
responses[-1]

{'dets': array([-0.02960647,  0.05640502, -0.0231486 ,  0.02547812,  0.05955626,
        -0.07495055, -0.0481547 , -0.00211531, -0.00478945,  0.09500429,
         0.00023158,  0.0686745 ,  0.07587979,  0.01008717, -0.12256154,
         0.0587662 ], dtype=float32),
 'idx': [array([0, 1, 2, 3, 4, 5, 8, 9], dtype=uint8),
  array([0, 1, 2, 3, 4, 5, 7, 9], dtype=uint8),
  array([0, 1, 2, 3, 4, 5, 7, 8], dtype=uint8),
  array([0, 1, 2, 3, 4, 5, 6, 9], dtype=uint8),
  array([0, 1, 2, 3, 4, 5, 6, 8], dtype=uint8),
  array([0, 1, 2, 3, 4, 5, 6, 7], dtype=uint8),
  array([0, 1, 2, 3, 4, 5, 6, 7, 8], dtype=uint8),
  array([0, 1, 2, 3, 4, 5, 6, 7, 9], dtype=uint8),
  array([0, 1, 2, 3, 4, 5, 6, 8, 9], dtype=uint8),
  array([0, 1, 2, 3, 4, 5, 7, 8, 9], dtype=uint8),
  array([0, 1, 2, 3, 4, 6, 7, 8, 9], dtype=uint8),
  array([0, 1, 2, 3, 5, 6, 7, 8, 9], dtype=uint8),
  array([0, 1, 2, 4, 5, 6, 7, 8, 9], dtype=uint8),
  array([0, 1, 3, 4, 5, 6, 7, 8, 9], dtype=uint8),
  array([0, 2, 3, 4, 5, 6, 7, 8,